# Logistic Regression Training

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import re
import pandas as pd

### Load code for this project

In [ ]:
import logistic.train
import logistic.eval
%load_ext autoreload
%autoreload 1
%aimport logistic.train
%aimport logistic.eval

In [ ]:
%%time
!rm -rf cnn/logs/*
logistic.train.train(data_dir='./data_processing/voxels/', log_dir='logistic/logs/', examples_limit=20,
                solver='newton-cg', #{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}
                penalty='l2', 
                regularization_strength=10,
                cv_fold=5,
                max_iter=1000,
                tol=1e-4
               )

In [ ]:
logistic.eval.eval(model_file='logistic/logs/model.p', data_dir='./data_processing/voxels/')